In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MinMaxScaler

In [3]:
en = pd.read_csv("/content/drive/MyDrive/siqi mock/csv files/final_all_lyrics_negpos.csv", encoding='utf-8')

In [4]:
en

,artist,song,lyrics,bert_nega,bert_posi,bert_score,nltk_neg,nltk_neu,nltk_pos,nltk_score
0,Miley Cyrus,Flowers,"We were good, we were gold Kind of dream that ...",13,9,-0.144740,0,15,7,0.238127
1,SZA,Kill Bill,I'm still a fan even though I was salty Hate t...,15,7,-0.207048,2,16,4,0.054209
2,"Metro Boomin, The Weeknd, 21 Savage",Creepin' (with The Weeknd & 21 Savage),Oooh Just can't believe this man (Metro Boomin...,17,7,-0.239239,0,24,0,0.047654
3,"Metro Boomin, Future, Chris Brown",Superhero (Heroes & Villains) [with Future & C...,"(Metro, metro, metro, metro) Drankin' dope tur...",28,5,-0.427365,1,30,2,-0.013979
4,"Drake, 21 Savage",Rich Flex,"Go buy a zip of weed, hit the club Pay for 'bo...",18,10,-0.190593,1,25,2,-0.034607
...,...,...,...,...,...,...,...,...,...,...
241,"Grupo Marca Registrada, Grupo Frontera",Di Que Si,He was a boy who was a dreamer And he flew so ...,10,2,-0.383420,0,12,0,-0.024825
242,Mae Stephens,If We Ever Broke Up,"Throw it, throw it up Watch it all fall out Po...",17,16,-0.042716,0,33,0,0.002339
243,"$uicideboy$, Shakewell",Big Shot Cream Soda,"Rain, he wanted it comfortable I wanted that p...",10,10,-0.011087,2,13,5,0.135535
244,Chino Pacas,El Gordo Trae El Mando,P-P-P-P-P-PARTY That smile on your face Makes ...,22,4,-0.432062,1,19,6,0.001535


#hit / nohit (100)

In [4]:
dataset = pd.read_csv("/content/drive/MyDrive/siqi mock/csv files/final_all_data.csv", encoding = 'utf-8')

In [5]:
rank_list = dataset["rank"].values.tolist()
top100 = []

for rank in rank_list:
    if rank < 101:
        top100.append(1)
    else:
        top100.append(0)

dataset["top100"] = top100
dataset

,rank,uri,artist_names,track_name,source,peak_rank,previous_rank,weeks_on_chart,streams,danceability,...,duration_ms,time_signature,bert_nega,bert_posi,bert_score,nltk_neg,nltk_neu,nltk_pos,nltk_score,top100
0,1,spotify:track:0yLdNVWF3Srea0uzk55zFn,Miley Cyrus,Flowers,Columbia,1,1,3,18441075,0.707,...,200455,4,13.0,9.0,-0.144740,0.0,15.0,7.0,0.238127,1
1,2,spotify:track:1Qrg8KqiBpW07V7PNxwwwL,SZA,Kill Bill,Top Dawg Entertainment/RCA Records,1,2,8,15313761,0.644,...,153947,4,15.0,7.0,-0.207048,2.0,16.0,4.0,0.054209,1
2,3,spotify:track:2dHHgzDwk4BJdRwy9uXhTO,"Metro Boomin, The Weeknd, 21 Savage",Creepin' (with The Weeknd & 21 Savage),Republic Records,1,3,9,8185172,0.715,...,221520,4,17.0,7.0,-0.239239,0.0,24.0,0.0,0.047654,1
3,4,spotify:track:0vjeOZ3Ft5jvAi9SBFJm1j,"Metro Boomin, Future, Chris Brown",Superhero (Heroes & Villains) [with Future & C...,Republic Records,2,4,9,7350116,0.526,...,182667,4,28.0,5.0,-0.427365,1.0,30.0,2.0,-0.013979,1
4,5,spotify:track:1bDbXMyjaUIooNwFE9wn0N,"Drake, 21 Savage",Rich Flex,OVO / Republic Records,1,5,13,7109310,0.561,...,239360,3,18.0,10.0,-0.190593,1.0,25.0,2.0,-0.034607,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,196,spotify:track:01Lr5YepbgjXAWR9iOEyH1,Chief Keef,Love Sosa,UMGRI Interscope,44,178,55,1921806,0.772,...,246213,4,18.0,2.0,-0.459613,4.0,12.0,4.0,-0.122535,0
796,197,spotify:track:5OELUCYgOHKFAvCERnAvfS,Morgan Wallen,More Than My Hometown,Big Loud / Republic,21,-1,99,1902147,0.620,...,216573,4,29.0,13.0,-0.236389,1.0,39.0,2.0,0.047281,0
797,198,spotify:track:58k32my5lKofeZRtIvBDg9,Baby Keem,HONEST,Baby Keem LLC,39,196,29,1900952,0.676,...,172672,4,13.0,11.0,-0.056934,1.0,22.0,1.0,0.028050,0
798,199,spotify:track:7yq4Qj7cqayVTp3FF9CWbm,Vance Joy,Riptide,F-Stop Records/Atlantic,95,-1,26,1893077,0.484,...,204280,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
